#TASK 1.1
Write a function that computes the total buy volume for flex power, another that computes the total sell volume.

In [3]:
def compute_total_buy_volume(sql_database):
    import sqlite3
    import pandas as pd
    with sqlite3.connect(sql_database) as conn:
        df = pd.read_sql_query("SELECT * FROM epex_12_20_12_13", conn)
    total_buy_volume = df[df['side'] == 'buy']['quantity'].sum()
    print("Total Buy Volume:", total_buy_volume)

def compute_total_sell_volume(sql_database):
    import sqlite3
    import pandas as pd
    with sqlite3.connect(sql_database) as conn:
        df = pd.read_sql_query("SELECT * FROM epex_12_20_12_13", conn)
    total_sell_volume = df[df['side'] == 'sell']['quantity'].sum()
    print("Total Sell Volume:", total_sell_volume)


In [5]:
compute_total_buy_volume("trades.sqlite")
compute_total_sell_volume("trades.sqlite")

Total Buy Volume: 28
Total Sell Volume: 53


#Task 1.2:
Write a function that computes the PnL (profit and loss) of each strategy in euros. It's defined as the sum of the incomes realized with each trade.

If we sell energy, our income is quantity * price since we got money for our electricity. If we buy energy, our income is -quantity * price.

In [10]:
def compute_pnl(strategy, sql_database):
    from datetime import datetime, timezone
    import sqlite3
    import pandas as pd

    with sqlite3.connect(sql_database) as conn:
        df = pd.read_sql_query("SELECT * FROM epex_12_20_12_13", conn)


    # Filter trades by strategy
    df = df[df['strategy'] == strategy]

    if df.empty:
        print(f"No trades found for {strategy}")
        return 0.0
    
    df['income'] = df.apply(
        lambda row: row['quantity'] * row['price'] if row['side'] == 'sell'
        else -row['quantity'] * row['price'], axis=1
    )

    # Compute total PnL
    total_pnl = float(df['income'].sum())

    return {
        "strategy":     strategy,
        "value":        total_pnl,
        "unit":         "euro",
        "capture_time": datetime.now(timezone.utc).isoformat() + "Z"
    }



In [11]:
compute_pnl("strategy_1", "trades.sqlite")

{'strategy': 'strategy_1',
 'value': -50.0,
 'unit': 'euro',
 'capture_time': '2025-04-23T13:48:26.546491+00:00Z'}

Task 1.3:
Expose the function defined in the second task as an entrypoint of a web application.